In [ ]:
import numpy as np
from scipy.linalg import eig 
import mdptoolbox, mdptoolbox.example
import random
from numpy.random import seed
import time


def get_next_state(state,action):
    if state == 0:
        if action == 0:
            next_state = 0
            reward = 1
        else:
            next_state = 1
            reward = 1
    if state == 1:
        if action == 0:
            next_state = 0
            reward = 1
        else:
            next_state = 2
            reward = 1
    if state == 2:
        if action == 0:
            next_state = 1
            reward = 1
        else:
            next_state = 2
            reward = 1

    return next_state, reward
    
    
def feature_state(thetha,state):
    if state == 0:
        return thetha[0]
    if state == 1:
        return thetha[0]+ thetha[1]
    if state == 2:
        return thetha[1]
    
def grad_state(state):
    if state == 0:
        return [1,0]
    if state == 1:
        return [1,1]
    if state == 2:
        return [0,1]
    

total_runs = 10
total_iterations = 1000000 #500000
step_size = 0.00001

error_storage_td = np.zeros((total_runs,total_iterations))
error_storage_ptd = np.zeros((total_runs,total_iterations))
error_storage_etd = np.zeros((total_runs,total_iterations))

opt = np.array([[10,10,10]])

for runs in range(total_runs):
    
    np.random.seed((runs+1)*100)
    random.seed((runs+1)*110)

    pi = np.array([[0,1],[0.5,0.5],[1,0]])
    mu = np.array([[0.9,0.1],[0.5,0.5],[0.1,0.9]])
    d_mu = np.array([[5/11,1/11,5/11]])
    thetha = [0,0]
    discount = 0.9
    P_pi= np.array([[0,1,0],[0.5,0,0.5],[0,1,0]])

    F=1
    thetha_ETD = [0,0]
    thetha_td = [0,0]

    eta = 0.5
    state = 0


    for main_iter in range(total_iterations):

        action = np.random.choice(2,1,p=mu[state])[0]
        importance_sampling = pi[state,action]/mu[state,action]     
        next_state,reward = get_next_state(state,action)

        grad = grad_state(state)
        delta = reward + discount*feature_state(thetha_ETD,next_state) - feature_state(thetha_ETD,state)
        temp = F*importance_sampling*step_size*delta
        thetha_ETD = thetha_ETD + np.multiply(temp,grad)

        delta = reward + discount*feature_state(thetha,next_state) - (1+eta)*feature_state(thetha,state)
        temp2 = importance_sampling*step_size*delta
        thetha = thetha + np.multiply(temp2,grad)
        
        delta = reward + discount*feature_state(thetha_td,next_state) - feature_state(thetha_td,state)
        temp3 = importance_sampling*step_size*delta
        thetha_td = thetha_td + np.multiply(temp3,grad)

        F = 1+ discount*importance_sampling*F 
        state = next_state
        
        v_etd = np.zeros((3,1))
        v_ptd = np.zeros((3,1))
        v_td  = np.zeros((3,1))

        for i in range(3):
            v_ptd[i,0] = feature_state(thetha,i)
            v_etd[i,0] = feature_state(thetha_ETD,i)
            v_td[i,0] = feature_state(thetha_td,i)
            
        error_storage_ptd[runs,main_iter] = np.sqrt(np.sum(np.multiply(d_mu.T,np.square(opt.T - v_ptd))))
        error_storage_etd[runs,main_iter] = np.sqrt(np.sum(np.multiply(d_mu.T,np.square(opt.T - v_etd))))
        error_storage_td[runs,main_iter] = np.sqrt(np.sum(np.multiply(d_mu.T,np.square(opt.T - v_td))))

print(thetha,thetha_ETD,thetha_td)